---
title: "Generative AI: Chapter 3"
description: "Variational Autoencoders"
date: "2024-02-28"
# image: "deep_learning_model.png"
categories: [AI, Engineering, GenerativeAI]
format:
  html:
    code-fold: true
---

# Variational Autoencoders
These are notes from chapter 3 of Generative Deep Learning by David Foster.

::: {.callout-tip title="Story Time"}
Imagine an infinite wardrobe organised by "type" of clothing. 

Shoes would be close together, but formal shoes might be closer to the suits and trainers closer to the sports gear. Shirts and t-shirts would be close together. Coats might be nearby; the shirt->coat vector applied to t-shirts might lead you to "invent" gilets.

This encapsulates the idea of using a lower dimensional (2D in this case) latent space to **encode** the representation of more complex objects.

We could *sample* from some of the empty spaces to invent new hybrids of clothing. This generative step is **decoding** the latent space.
:::


## 1. Autoencoders
The idea of autoencoders (read: self-encoders) is that they learn to simplify the input then reconstruct it; the input and target output are the same.

- The **encoder** learns a lower dimensional representation of the image called the *embedding*.
- The **decoder** takes an embedding and recreates a higher-dimensional image. This should be an accurate reconstruction of the input.

This can be used as a generative model because we can the sample and decode *new* points from the latent space to generate novel outputs.